In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()
print("Current Directory:", current_dir)

# Change the current working directory to the parent directory
parent_dir = os.path.dirname(current_dir)#
os.chdir(parent_dir)

In [ ]:
import sys, os
# add code root to python path
sys.path.append('code/')
import numpy as np
# from eval.eval_contrastive_scott import ContrastiveAnswerEvaluator
from core.utils import ResourceManager
from core.models import CostManager
from core.card import GenerativeCard
from core.data import MMLUBatch, load_mmlu_batches
from core.data import load_batches as lb
import time
from core.config import *
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm, trange
import json
from openai import OpenAI

In [ ]:
paraphrase = True

In [ ]:
def load_batches(topic, models, source='test'):
    test = source == 'test'
    all_test_batches = {}
    for model in models:
        batch = lb(
                "../datasets/gsm8k/gsm8k", topic, model, source, [60 if test else 40], False
            )[0]
        
        all_test_batches[model] = batch

    return all_test_batches


def get_latest_folder(topic, optim_method, card_format, evaluator, model, generation_method='generative'):
    folder_root = f'../outputs/{generation_method}/{topic}/{optim_method}/{card_format}/{evaluator}/{model}'
    all_folders =  os.listdir(f'../outputs/{generation_method}/{topic}/{optim_method}/{card_format}/{evaluator}/{model}')
    all_folders.sort()
    all_folders = all_folders[::-1]
    for folder in all_folders:
        if re.match(r"\d{2}-\d{2}_\d{2}-\d{2}-\d{2}_" , folder):
            return f'{folder_root}/{folder}' 
    return None

def load_cards(topic,
               card_format,
               models,
               evaluator,
               method='contrastive'):
    
    cards = {}
    for model in models:
        path = get_latest_folder(topic, 'prog-reg', card_format, evaluator, model, 'generative')
        with open(path+'/cards/epoch_4_card.json', 'r') as f:
            cards[model] = str(GenerativeCard(d= json.load(f)))

    return cards

def get_all_completions(batch, paraphrase=False):
    if not paraphrase:
        return [batch.get_model_reasoning(i) for i in range(len(batch))]
    else:
        return [batch.get_paraphrased(i) for i in range(len(batch))]

def embed_helper(index, completion, model):
    client = OpenAI(api_key=OPENAI_API_KEY)
    completions = completion.replace("\n", " ")
    return client.embeddings.create(input = [completions], 
                                    model=model).data[0].embedding

def get_all_embeddings(completions, model="text-embedding-3-small"):
    with ThreadPoolExecutor(max_workers=60) as executor:
        embeddings = list(tqdm(executor.map(embed_helper, range(len(completions)), completions, [model]*len(completions)), total=len(completions)))
    return embeddings


In [ ]:
# models = [
#     "gemma-1.1-7b-it",
#     "gpt-3.5-turbo",
#     "gpt-4-turbo",
#     "Meta-Llama-3-8B-Instruct",
#     "Meta-Llama-3-70B-Instruct",
#     "Mistral-7B-Instruct-v0.2",
#     "Mixtral-8x7B-Instruct-v0.1",
# ]

models = ['gpt-4o',
          'Meta-Llama-3-8B-Instruct',
          'Meta-Llama-3-70B-Instruct',
          'Mixtral-8x7B-Instruct-v0.1',
          'Mistral-7B-Instruct-v0.2',]

topic = 'gsm8k'

evaluator = 'gpt'

In [ ]:
all_batches = load_batches(topic, models)
# all_cards = load_cards(topic, 'dict', models, evaluator)
all_completions = {k: get_all_completions(v, paraphrase) for k,v in all_batches.items()}

In [ ]:
all_embeddings = {}
for model in models:
    all_embeddings[model] = get_all_embeddings(all_completions[model])

In [ ]:
# save embeddings
# make director if not exist

for model in models:
    if not os.path.exists(f'../code/embeddings/mmlu/{topic}'):
        os.makedirs(f'../code/embeddings/mmlu/{topic}')

    np.save(f'../code/embeddings/mmlu/{topic}/{model}{"-paraphrase" if paraphrase else ""}-test.npy', all_embeddings[model])

In [ ]:
all_batches = load_batches(topic, models, 'train')
# all_cards = load_cards(topic, 'dict', models, evaluator)
all_completions = {k: get_all_completions(v) for k,v in all_batches.items()}

In [ ]:
all_embeddings = {}
for model in models:
    all_embeddings[model] = get_all_embeddings(all_completions[model])

In [ ]:
# save embeddings
# make director if not exist

for model in models:
    if not os.path.exists(f'../code/embeddings/mmlu/{topic}'):
        os.makedirs(f'../code/embeddings/mmlu/{topic}')

    np.save(f'../code/embeddings/mmlu/{topic}/{model}{"-paraphrase" if paraphrase else ""}-train.npy', all_embeddings[model])

In [ ]:
print(len(all_completions[models[0]]))